In [ ]:
# Step 1: Install transformers and torch
!pip install transformers torch --quiet

# Step 2: Import required libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--- Story continuation #1 ---
Once upon a time in a distant kingdom, there was a young princess who dreamed of adventure. One day, she decided to leave the castle and explore the mysterious forest nearby.One day, she ended up in a strange place. In the forest, she met a beautiful young boy named Alucard. When the young boy asked her if she could become his apprentice, she accepted and began her journey to learn the skills that would eventually lead her to becoming a Knight.

--- Story continuation #2 ---
Once upon a time in a distant kingdom, there was a young princess who dreamed of adventure. One day, she decided to leave the castle and explore the mysterious forest nearby.However, she soon realized that she was not alone. She soon discovered that the forest was full of monsters. The Princess and her friends would soon encounter more monsters. Soon after, the princess and her friends discovered that there was a powerful monster in the forest. This monster was called "The Evil Monster

In [ ]:
# Step 3: Load pre-trained GPT-2 tokenizer and model
model_name = "gpt2-medium"  # You can use "gpt2", "gpt2-medium", "gpt2-large" or "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Put model in evaluation mode and move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Step 4: Define a function to generate story continuation
def generate_story(prompt, max_length=200, temperature=0.7, top_k=50, top_p=0.95, num_return_sequences=1):
    """
    Generate story continuation using GPT-2.

    Args:
        prompt (str): The initial text prompt to start the story.
        max_length (int): Maximum length of generated text including prompt.
        temperature (float): Controls randomness in generation (higher = more random).
        top_k (int): Limits sampling to top_k tokens.
        top_p (float): Nucleus sampling probability threshold.
        num_return_sequences (int): Number of generated sequences to return.

    Returns:
        List of generated story continuations.
    """
    # Encode input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate output sequences
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return generated texts
    generated_stories = []
    for generated_sequence in output_sequences:
        text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        # Remove the prompt from the output to get only the continuation
        continuation = text[len(prompt):].strip()
        generated_stories.append(continuation)

    return generated_stories

# Step 5: Example usage
if __name__ == "__main__":
    prompt = (
        "Once upon a time in a distant kingdom, there was a young princess who dreamed of adventure. "
        "One day, she decided to leave the castle and explore the mysterious forest nearby."
    )
    stories = generate_story(prompt, max_length=250, temperature=0.8, top_k=40, top_p=0.9, num_return_sequences=2)

    for i, story in enumerate(stories, 1):
        print(f"--- Story continuation #{i} ---\n{prompt}{story}\n")

--- Story continuation #1 ---
Once upon a time in a distant kingdom, there was a young princess who dreamed of adventure. One day, she decided to leave the castle and explore the mysterious forest nearby.But as she passed through the forest, she suddenly noticed that something was wrong. A beautiful, green, glowing, glowing dragon appeared out of nowhere. The princess was terrified and rushed back to the castle, but she soon realized that the dragon was actually her father, King Razzo. The prince was furious, and attacked the dragon, but was stopped by Razzo. The princess realized that she and her father were not meant to be together, and that she should tell Razzo everything about herself. The princess took Razzo's sword and threw it into the dragon's stomach, causing it to go into a rage and attack the princess. The princess told Razzo that she wanted to kill Razzo and her father, and to do so, she needed to find out who her father was. The princess told Razzo that she was going to t